In [1]:
# 실습에 필요한 라이브러리를 불러옵니다.
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# MNIST DATASET 다운로드
!git clone https://github.com/baek2sm/ml.git
!tar -zxvf ./ml/datasets/MNIST.tar.gz

# 현재 경로에 MNIST 학습 세트와 테스트 세트를 불러옵니다.
path = './'
train_dataset = datasets.MNIST(path, train=True, download=True)
test_dataset = datasets.MNIST(path, train=False, download=True)

# 학습 세트와 테스트 세트의 입력 데이터와 타깃을 준비합니다.
X_train, y_train = train_dataset.data / 255, train_dataset.targets
X_test, y_test = test_dataset.data / 255, test_dataset.targets

Cloning into 'ml'...
x MNIST/
x MNIST/raw/
x MNIST/raw/train-labels-idx1-ubyte
x MNIST/raw/t10k-labels-idx1-ubyte.gz
x MNIST/raw/t10k-labels-idx1-ubyte
x MNIST/raw/t10k-images-idx3-ubyte.gz
x MNIST/raw/train-images-idx3-ubyte
x MNIST/raw/train-labels-idx1-ubyte.gz
x MNIST/raw/t10k-images-idx3-ubyte
x MNIST/raw/train-images-idx3-ubyte.gz
x MNIST/processed/
x MNIST/processed/training.pt
x MNIST/processed/test.pt


In [3]:
# 학습 세트와 테스트 세트의 데이터 형태를 확인합니다.
print('학습 세트 입력 데이터:', X_train.shape)
print('학습 세트 타깃:', y_train.shape)
print('테스트 세트 입력 데이터:', X_test.shape)
print('테스트 세트 타깃:', y_test.shape)

학습 세트 입력 데이터: torch.Size([60000, 28, 28])
학습 세트 타깃: torch.Size([60000])
테스트 세트 입력 데이터: torch.Size([10000, 28, 28])
테스트 세트 타깃: torch.Size([10000])


In [4]:
# 2차원(높이, 너비) 형태인 이미지 데이터를 3차원(채널, 높이, 너비) 형태로 변환합니다.
X_train, X_test = X_train.unsqueeze(1), X_test.unsqueeze(1)

# 학습 세트와 테스트 세트의 데이터 형태를 확인합니다.
print('학습 세트 입력 데이터:', X_train.shape)
print('테스트 세트 입력 데이터:', X_test.shape)

학습 세트 입력 데이터: torch.Size([60000, 1, 28, 28])
테스트 세트 입력 데이터: torch.Size([10000, 1, 28, 28])


In [5]:
# 입력 데이터와 타깃을 묶어 텐서 데이터세트를 생성합니다.
train_dset = TensorDataset(X_train, y_train)
test_dset = TensorDataset(X_test, y_test)

# 한 번에 128개의 데이터 샘플을 배치로 사용하는 데이터로더를 생성합니다.
train_loader = DataLoader(train_dset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=False)

In [6]:
# CNN 모델 클래스를 정의합니다.
class CNN(nn.Module):
    # 생성자에서 모델의 구조를 정의합니다. 
    def __init__(self):
        # 상속받아 생성한 객체이므로 부모(nn.Module)의 생성자를 호출합니다.
        super().__init__()
        # 첫 번째 은닉층을 정의합니다.
        self.hidden_layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(0.5)      
        )        
        # 두 번째 은닉층을 정의합니다.
        self.hidden_layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)),      
            nn.Dropout(0.5)      
        )
        # 세 번째 은닉층을 정의합니다.
        self.hidden_layer3 = nn.Linear(128*5*5, 128)
        # 출력층을 정의합니다.
        self.output_layer = nn.Linear(128, 10)        
    # 모델의 순전파를 정의합니다.
    def forward(self, X):
        # 생성자에서 만든 은닉층과 출력층 노드로 타깃을 추론하고 반환합니다.
        out = self.hidden_layer1(X)
        out = self.hidden_layer2(out)                        
        out = out.view(out.shape[0], -1)  # 전결합층을 사용하기 위해 1차원 배열 형태로 변환합니다.        
        out = self.hidden_layer3(out)
        out = self.output_layer(out)
        return out

In [7]:
# 그래픽카드 사용이 가능할 경우 그래픽카드로 연산하도록 설정합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 합성곱 신경망 모델 객체를 생성합니다.
model = CNN().to(device)

# 크로스 엔트로피(Cross Entropy) 손실 함수 객체를 생성합니다.
criterion = nn.CrossEntropyLoss()

# 확률적 경사 하강법 옵티마이저 객체를 생성합니다.
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [8]:
# 학습 함수를 정의합니다.
def train(model, criterion, optimizer, loader):
  # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
  epoch_loss = 0
  epoch_acc = 0

  # 모델을 학습 모드로 설정합니다.
  model.train()

  # 배치 학습을 실행합니다.
  for X_batch, y_batch in loader:
    # 입력 데이터와 타깃의 배치를 그래픽 카드로 연산하도록 설정합니다.
    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
    # 기울기를 초기화합니다.
    optimizer.zero_grad()
    # 모델을 사용해 타깃을 추론합니다.
    hypothesis = model(X_batch)
    # 손실 함수로 오차를 계산합니다.
    loss = criterion(hypothesis, y_batch)        
    # 기울기를 계산합니다.
    loss.backward()
    # 경사 하강법으로 가중치를 수정합니다.
    optimizer.step()
    # 정확도를 계산합니다.
    y_predicted = torch.argmax(hypothesis, 1)
    acc = (y_predicted == y_batch).float().mean()
    # 현재 배치의 오차와 정확도를 저장합니다.
    epoch_loss += loss.item()
    epoch_acc += acc.item()

  # 현재 에포크의 오차와 정확도를 반환합니다.
  return epoch_loss / len(loader), epoch_acc / len(loader)

In [9]:
# 평가 함수를 정의합니다.
def evaluate(model, criterion, loader):
  # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
  epoch_loss = 0
  epoch_acc = 0

  # 모델을 평가 모드로 설정합니다.
  model.eval()

  with torch.no_grad():
    # 배치 단위로 추론을학습을 실행합니다.
    for X_batch, y_batch in loader:
      # 입력 데이터와 타깃의 배치를 그래픽 카드로 연산하도록 설정합니다.
      X_batch, y_batch = X_batch.to(device), y_batch.to(device)
      # 모델을 사용해 타깃을 추론합니다.
      hypothesis = model(X_batch)
      # 손실 함수로 오차를 계산합니다.
      loss = criterion(hypothesis, y_batch)
      # 정확도를 계산합니다.
      y_predicted = torch.argmax(hypothesis, 1)
      acc = (y_predicted == y_batch).float().mean()
      # 현재 배치의 오차와 정확도를 저장합니다.
      epoch_loss += loss.item()
      epoch_acc += acc.item()

    # 현재 에포크의 오차와 정확도를 반환합니다.
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [10]:
# 20에 걸쳐 모델을 학습시킵니다.
n_epochs = 20
for epoch in range(1, n_epochs+1):
  # 모델을 학습시킵니다.
  loss, acc = train(model, criterion, optimizer, train_loader)

  # 모델을 평가합니다.
  test_loss, test_acc = evaluate(model, criterion, test_loader)

  # 현재 에포크의 학습 결과를 출력합니다.
  print('epoch: {}, loss: {:.3f}, acc: {:.2f}, test_loss: {:.3f}, test_acc: {:.3f}'.format(
       epoch, loss, acc, test_loss, test_acc))


c:\users\home\anaconda3\envs\dev\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: 1, loss: 0.413, acc: 0.88, test_loss: 0.116, test_acc: 0.969
epoch: 2, loss: 0.140, acc: 0.96, test_loss: 0.073, test_acc: 0.979
epoch: 3, loss: 0.103, acc: 0.97, test_loss: 0.057, test_acc: 0.982
epoch: 4, loss: 0.086, acc: 0.97, test_loss: 0.047, test_acc: 0.985
epoch: 5, loss: 0.078, acc: 0.98, test_loss: 0.044, test_acc: 0.985
epoch: 6, loss: 0.069, acc: 0.98, test_loss: 0.039, test_acc: 0.988
epoch: 7, loss: 0.067, acc: 0.98, test_loss: 0.037, test_acc: 0.987
epoch: 8, loss: 0.060, acc: 0.98, test_loss: 0.034, test_acc: 0.989
epoch: 9, loss: 0.057, acc: 0.98, test_loss: 0.035, test_acc: 0.989
epoch: 10, loss: 0.054, acc: 0.98, test_loss: 0.034, test_acc: 0.988
epoch: 11, loss: 0.052, acc: 0.98, test_loss: 0.031, test_acc: 0.989
epoch: 12, loss: 0.049, acc: 0.98, test_loss: 0.032, test_acc: 0.989
epoch: 13, loss: 0.047, acc: 0.98, test_loss: 0.030, test_acc: 0.990
epoch: 14, loss: 0.045, acc: 0.99, test_loss: 0.029, test_acc: 0.990
epoch: 15, loss: 0.043, acc: 0.99, test_los